In [ ]:
pd.set_option('display.max_rows', 1000)

In [23]:
import pandas as pd
import copy
import numpy as np
import scipy as sp
import statistics
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
results = pd.read_csv("~/Downloads/REAL-TIME/results.csv", sep=';', header=0, encoding='latin-1')
cycles = pd.read_csv("~/Downloads/REAL-TIME/cycles.csv", sep=',', header=0)
cycles = cycles[["Target", "Sample", "Biological Set Name", "Cq", "Cq Mean", "Cq Std. Dev", "SQ Std. Dev"]]
cycles['Index'] = cycles[["Target", "Biological Set Name", "Sample"]].apply(lambda row: '_'.join(row.dropna()), axis=1)
#cycles = cycles.set_index(keys = 'Index', drop = False)
E = 2

In [3]:
cq = {}
for index, row in cycles.iterrows():
    target = cycles.loc[index].values[0]
    sample = cycles.loc[index].values[1]
    bioset = cycles.loc[index].values[2]
    cycle = cycles.loc[index].values[3]
    if target not in cq:
        cq[target] = {}
    if bioset not in cq[target]:
        cq[target][bioset] = {}
    if sample not in cq[target][bioset]:
        cq[target][bioset][sample] = list() 
    cq[target][bioset][sample].append(cycle)
    
cq_mean = copy.deepcopy(cq)
for target in cq_mean.keys():
    for bioset in cq_mean[target].keys():
        for sample in cq_mean[target][bioset].keys():
            cq_mean[target][bioset][sample] = statistics.mean(cq_mean[target][bioset][sample])
    
cq_std = copy.deepcopy(cq)
for target in cq_std.keys():
    for bioset in cq_std[target].keys():
        for sample in cq_std[target][bioset].keys():
            cq_std[target][bioset][sample] = statistics.stdev(cq_std[target][bioset][sample])

In [4]:
# Control Sample - NCA_-

cq_control = cq_mean['CHASERR']['NCA']['-']
cq_control_ref = cq_mean['PPIA']['NCA']['-']
relative_quantity = copy.deepcopy(cq_mean)

for bioset in relative_quantity['CHASERR'].keys():
    for sample in relative_quantity['CHASERR'][bioset].keys():
        cq_sample = cq_mean['CHASERR'][bioset][sample]
        relative_quantity['CHASERR'][bioset][sample] = E**(cq_control-cq_sample)

for bioset in relative_quantity['PPIA'].keys():
    for sample in relative_quantity['PPIA'][bioset].keys():
        cq_sample = cq_mean['PPIA'][bioset][sample]
        relative_quantity['PPIA'][bioset][sample] = E**(cq_control_ref-cq_sample)
        
relative_quantity_std = copy.deepcopy(cq)
for target in relative_quantity_std.keys():
    for bioset in relative_quantity_std[target].keys():
        for sample in relative_quantity_std[target][bioset].keys():
            relative_quantity_std[target][bioset][sample] = cq_std[target][bioset][sample] * relative_quantity[target][bioset][sample] * np.log(E)

In [28]:
norm_exp = copy.deepcopy(relative_quantity)

for bioset in norm_exp['CHASERR'].keys():
    for sample in norm_exp['CHASERR'][bioset].keys():
        norm_exp['CHASERR'][bioset][sample] = relative_quantity['CHASERR'][bioset][sample] / relative_quantity['PPIA'][bioset][sample]
del norm_exp['PPIA']

norm_exp_std = copy.deepcopy(norm_exp)
for bioset in norm_exp_std['CHASERR'].keys():
    for sample in norm_exp_std['CHASERR'][bioset].keys():
        norm_exp_std['CHASERR'][bioset][sample] = ((relative_quantity_std['PPIA'][bioset][sample] / relative_quantity['PPIA'][bioset][sample])**2 + (relative_quantity_std['CHASERR'][bioset][sample] / relative_quantity['CHASERR'][bioset][sample])**2)**0.5

In [43]:
names = []
sep = '_'
for bioset in norm_exp['CHASERR'].keys():
    for sample in norm_exp['CHASERR']['KD'].keys():
        expr = norm_exp['CHASERR']['KD'][sample]
        names.append(bioset+sep+sample)        
names

['KD_AG',
 'KD_-',
 'KD_MTX',
 'KD_CGP',
 'KD_ABT',
 'KD_17DM',
 'KD_CPA',
 'KD_Cycl',
 'KD_DMSO',
 'NCA_AG',
 'NCA_-',
 'NCA_MTX',
 'NCA_CGP',
 'NCA_ABT',
 'NCA_17DM',
 'NCA_CPA',
 'NCA_Cycl',
 'NCA_DMSO']

In [61]:
expression = pd.DataFrame(data= {'ID': names})
expression[['Bioset', 'Sample']] = expression['ID'].str.split('_',expand=True)
expression['Expression'] = 0
expression['Expression_std'] = 0
for index, row in expression.iterrows():
    bioset = expression.loc[index].values[1]
    sample = expression.loc[index].values[2]
    expression.at[index, 'Expression'] = norm_exp['CHASERR'][bioset][sample]
    expression.at[index, 'Expression_std'] = norm_exp_std['CHASERR'][bioset][sample]

In [62]:
expression

,ID,Bioset,Sample,Expression,Expression_std
0,KD_AG,KD,AG,0.024091,0.136097
1,KD_-,KD,-,0.012143,0.283125
2,KD_MTX,KD,MTX,0.002191,0.437997
3,KD_CGP,KD,CGP,0.047077,0.068585
4,KD_ABT,KD,ABT,0.013513,0.287022
5,KD_17DM,KD,17DM,0.022346,0.036092
6,KD_CPA,KD,CPA,0.018841,0.249288
7,KD_Cycl,KD,Cycl,0.035307,0.275763
8,KD_DMSO,KD,DMSO,0.016212,0.050397
9,NCA_AG,NCA,AG,0.971461,0.031341


In [ ]:
sns.boxplot(data=expression, orient = 'v', x="age", y="class")

In [56]:
expression.loc[3].values[3] = 3

In [ ]:
# nu = Number of experimental replicas + number of control replicas - 2
# nu = 3
# t = statistics.mean(?) - statistics.mean(?)

In [20]:
A = sp.special.gamma((nu+1)/2) / (((nu* np.pi)**0.5) *  sp.special.gamma(nu/2)) * (1 + t**2)**((nu+1)/2)